In [90]:
import pandas as pd
import json
pd.set_option('display.max_colwidth', 100)

def json_dict_parse(file_path):
    data_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            record = json.loads(line.strip())
            data_list.append(record)
    print(f"读取了 {len(data_list)} 条记录")
    return data_list

def load_json_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        print(f"load json file from {path}")
        return json.load(f)

In [98]:
file_path = "../outputs/testA_data/output_spider.json"
data = json_dict_parse(file_path)
df = pd.DataFrame(data)
df.head(1)

读取了 1049 条记录


,idx,db_id,query,evidence,extracted_schema,ground_truth,difficulty,chosen_db_schem_dict,pruned,final_sql,qa_pairs,fixed,try_times,pred
0,155f9b1c-5502-4904-98e0-d57ba8e9bb43,中国城市,省花是玫瑰的是哪个省？,,{},,medium,"{'城市': ['城市id', '名称', '面积', '人口', '市花', '所属省id'], '市生产总值': ['年份', '城市id', 'gdp', '人均GDP'], '省份':...",False,SELECT `名称` FROM 省份 WHERE 省花 = '玫瑰',```sql\nSELECT `名称` FROM 省份 WHERE 省花 = '玫瑰'\n```,False,1,SELECT `名称` FROM 省份 WHERE 省花 = '玫瑰'


In [80]:
df_res = df[['idx', 'query', 'db_id', 'pred']]
df_res.columns = ["question_id", 'question', 'db_id', 'query']
# df_res['db_id'].value_counts()
df_res

,question_id,question,db_id,query
0,155f9b1c-5502-4904-98e0-d57ba8e9bb43,省花是玫瑰的是哪个省？,中国城市,SELECT `名称` FROM 省份 WHERE 省花 = '玫瑰'
1,032eee07-47b8-4e43-bde4-d40b4e879537,这个省有几个下辖区？,中国城市,"SELECT `名称`, `下辖区数量` FROM 省份 WHERE `名称` = '重庆市'"
2,fbb1c631-6965-4924-8a6e-fca05b19b680,它包含哪个城市？,中国城市,SELECT `名称` FROM 城市
3,dc7f9a9e-6a9e-4c5a-a4e9-24270c250e11,列出这个城市的人口与面积。,中国城市,"SELECT `人口`, `面积` FROM 城市"
4,4b84e019-6874-485b-bf3f-d26e2a62e25e,请问我想去逛超市应该去哪个商场呢？,购物城,SELECT 商场.`商场名` FROM 商场 JOIN 商家入驻商场 ON 商场.`商场id` = 商家入驻商场.`商场id` JOIN 商家 ON 商家入驻商场.`商家id` = 商家.`...
...,...,...,...,...
1028,87470869-ca91-4492-88d7-9df6933b896a,我想知道哪些寓言故事中的动物同时也是排名最聪明的动物。,DW,SELECT DW.`ZWXM`\nFROM DW\nJOIN DWFBCS ON DW.`DWID` = DWFBCS.`DWID`\nJOIN CS ON DWFBCS.`CSID` = ...
1029,f00ff87b-0c0e-4e57-8bfa-21d84fdff932,可能会被拍成电影的聪明动物有哪些？,DW,SELECT DISTINCT DW.`ZWXM`\nFROM DW\nINNER JOIN DWDY ON DW.`DWID` = DWDY.`DWID`\nWHERE DW.`SX` = ...
1030,76439cd3-5f8b-44da-9068-c120bc09eb95,在这些电影中，哪些是纪录片类型的？,DW,SELECT YPM FROM DWDY WHERE LX = '纪录片'
1031,ee22d4c0-ee8e-49ba-b6a6-ecea97d56a23,拍摄国家为四川的动物电影的类型。,DW,SELECT DISTINCT DWDY.`LX` \nFROM DWDY \nJOIN DW ON DWDY.`DWID` = DW.`DWID` \nJOIN DWFBCS ON DW.`...


## test验证

In [41]:
dev_data = load_json_file('../data/testA_data/dev.json')
df_dev = pd.DataFrame()
for items in dev_data:
    df_tmp = pd.DataFrame(items['interaction'])
    df_tmp['db_list'] = str(items['db_list'])
    df_dev= pd.concat([df_dev, df_tmp])
df_dev = df_dev.reset_index(drop=True)
df_dev['db_list'] = df_dev['db_list'].apply(eval)
df_dev['db_id'] = df_dev['db_list'].apply(lambda x:x[0])
df_dev

load json file from ../data/testA_data/dev.json


,question_id,question,db_list,db_id
0,155f9b1c-5502-4904-98e0-d57ba8e9bb43,省花是玫瑰的是哪个省？,[中国城市],中国城市
1,032eee07-47b8-4e43-bde4-d40b4e879537,这个省有几个下辖区？,[中国城市],中国城市
2,fbb1c631-6965-4924-8a6e-fca05b19b680,它包含哪个城市？,[中国城市],中国城市
3,dc7f9a9e-6a9e-4c5a-a4e9-24270c250e11,列出这个城市的人口与面积。,[中国城市],中国城市
4,4b84e019-6874-485b-bf3f-d26e2a62e25e,请问我想去逛超市应该去哪个商场呢？,[购物城],购物城
...,...,...,...,...
1100,06aa482f-ea81-45d7-84d7-9cf9a6c61979,这些动物分别分布在哪些城市？,"[DW, DWBD]",DW
1101,449aad39-153e-4e0e-af55-aba9891ca66b,这些城市的气候地带是什么？,"[DW, DWBD]",DW
1102,5a146ee8-37e0-45de-8f68-2de0c6c06fd3,哪种动物拥有最快的速度，并且它在每个城市的现存数量是多少？,"[DW, DWBD]",DW
1103,acdae273-4c92-4640-8f14-96998e8706ed,哪些城市拥有这种速度最快的动物？,"[DW, DWBD]",DW


In [43]:
# df_res.merge(df_dev, on=['question_id', 'question', 'db_id'])

In [104]:
df_sub1 = df_dev.merge(df_res.head(602), on=['question_id', 'question', 'db_id'], how='left')
# df_sub1 = df_sub1.fillna('NNNN')
df_sub1['query'] = df_sub1['query'].apply(lambda x:x.replace('`', ''))
df_sub1[df_sub1['query']!='NNNN']

AttributeError: 'float' object has no attribute 'replace'

In [74]:
sub_demo = load_json_file('../data/sub_demo.json')
for q_list in sub_demo:
    for item in q_list:
        row = df_sub1[df_sub1['question_id']==item['question_id']].iloc[0]
        item['db_id'] = row['db_id']
        item['query'] = row['query']

with open("../outputs/testA_data/sub0828.json", 'w', encoding='utf-8') as json_file:
    json.dump(sub_demo, json_file, ensure_ascii=False, indent=4)

load json file from ../data/sub_demo.json
